In [114]:
pip install pandas numpy opencv-python scikit-learn tensorflow keras

In [115]:
import os
import requests

bold text# Create a directory to store images locally

In [159]:
image_folder = 'images'
os.makedirs(image_folder, exist_ok=True)

In [160]:
# Function to download images from URLs and save locally


In [161]:
def download_images(image_links):
    for i, link in enumerate(image_links):
        try:
            response = requests.get(link)
            if response.status_code == 200:
                with open(f"{image_folder}/image_{i+1}.jpg", 'wb') as file:
                    file.write(response.content)
            else:
                print(f"Failed to download image {i+1}")
        except Exception as e:
            print(f"Error downloading image {i+1}: {e}")

In [162]:

 from google.colab import drive
 import pandas as pd

In [163]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [164]:
path = 'drive/My Drive/data'

In [165]:
train_data = pd.read_csv(path+'/COMBINED_TRAIN_DATA (1).csv', encoding='utf-8')



In [166]:
download_images(train_data['image_link'].tolist())

 Image Preprocessing: Resizing, Normalization, and Augmentation

Resize images to a consistent size (e.g., 128x128 pixels).
Normalize the pixel values (scale them between 0 and 1).
Optionally apply data augmentation (e.g., flipping, rotating, etc.).

In [167]:
pip install tensorflow opencv-python

In [168]:
import os
import numpy as np
import cv2  # OpenCV for image processing
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [169]:
TARGET_SIZE = (128, 128)


In [170]:
def preprocess_image(image_path, target_size=TARGET_SIZE):
    # Load the image using OpenCV
    img = cv2.imread(image_path)

    # Resize the image to the target size
    img = cv2.resize(img, target_size)

    # Normalize the image (scale pixel values to the range [0, 1])
    img = img.astype('float32') / 255.0

    return img

In [171]:
# Data Augmentation using Keras ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=20,        # Rotate images by up to 20 degrees
    width_shift_range=0.1,    # Shift the image width by up to 10%
    height_shift_range=0.1,   # Shift the image height by up to 10%
    horizontal_flip=True,     # Flip the images horizontally
    fill_mode='nearest'       # Fill in new pixels when shifting
)

In [172]:

# Function to augment a single image
def augment_image(img):
    img = np.expand_dims(img, axis=0)  # Expand dims for batch compatibility
    augmented_images = datagen.flow(img, batch_size=1)
    return augmented_images[0][0]  # Return the augmented image

 Load and Preprocess Images for Training and Testing

In [173]:
image_folder = 'images'

In [174]:
def load_and_preprocess_images(image_filenames, folder=image_folder, augment=False):
    images = []
    for filename in image_filenames:
        image_path = os.path.join(folder, filename)
        img = preprocess_image(image_path)

        # Optionally augment the image
        if augment:
            img = augment_image(img)

        images.append(img)

    return np.array(images)


In [191]:
train_data =  pd.read_csv(path+'/COMBINED_TRAIN_DATA (1).csv', encoding='utf-8')
test_data =  pd.read_csv(path+'/TEST.csv', encoding='utf-8')
T_D = train_data
TEST_DATA = test_data
print(T_D)
print(TEST_DATA)

    index                                         image_link  group_id  \
0       1  https://m.media-amazon.com/images/I/71Zy+qaX82...         3   
1       2  https://m.media-amazon.com/images/I/619xuNVks4...         4   
2       3  https://m.media-amazon.com/images/I/61+uFY1+-d...         4   
3       4  https://m.media-amazon.com/images/I/413lP0yIzL...         1   
4       5  https://m.media-amazon.com/images/I/41KG3OYJo0...         3   
5       6  https://m.media-amazon.com/images/I/81vqHyDCSC...         2   
6       7  https://m.media-amazon.com/images/I/71GLMJ7TQi...         2   
7       8  https://m.media-amazon.com/images/I/41uC6f6q+B...         2   
8       9  https://m.media-amazon.com/images/I/41aIgv8HTE...         2   
9      10  https://m.media-amazon.com/images/I/71fhaUR7VG...         2   
10     11  https://m.media-amazon.com/images/I/21q4I7ziVq...         2   
11     12  https://m.media-amazon.com/images/I/61MV+cJP9U...         6   
12     13  https://m.media-amazon.com/

In [192]:
# List of image filenames (as used in the earlier step)
train_image_filenames = [f'image_{i+1}.jpg' for i in range(train_data.shape[0])]
test_image_filenames = [f'image_{i+1}.jpg' for i in range(test_data.shape[0])]

In [193]:
X_train_images = load_and_preprocess_images(train_image_filenames, augment=True)  # With augmentation
X_test_images = load_and_preprocess_images(test_image_filenames)  # Without augmentation

Feeding Preprocessed Images into the Model

In [194]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Define a simple CNN model

In [195]:
def create_model(input_shape):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1))  # Output layer for regression (e.g., predicting dimensions or weight)
    return model

In [196]:
# Define input shape based on image size and color channels (3 for RGB)
input_shape = (TARGET_SIZE[0], TARGET_SIZE[1], 3)

In [197]:
# Create and compile the CNN model
model = create_model(input_shape)
model.compile(optimizer='adam', loss='mean_squared_error')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [198]:

import re

# Function to extract the numeric value from the string
def extract_numeric_value(entity_value):
    # Use a regular expression to find numeric values in the string
    match = re.search(r"[-+]?\d*\.\d+|\d+", entity_value)
    if match:
        # Return the numeric value as a float
        return float(match.group())
    else:
        return None

# Apply the function to the 'entity_value' column
train_data['numeric_value'] = train_data['entity_value'].apply(extract_numeric_value)

# Now, you can safely convert the extracted numeric values to float for your target variable
y_train = train_data['numeric_value']

# Check if there are any missing values in the new 'numeric_value' column
print(train_data['numeric_value'].isnull().sum())

0


In [199]:
# Drop rows with missing values (optional, depending on your task)
train_data.dropna(subset=['numeric_value'], inplace=True)

# Or fill missing values with a default (e.g., 0)
train_data['numeric_value'].fillna(0, inplace=True)

# Now you can use 'numeric_value' as your target variable
y_train = train_data['numeric_value']

In [200]:
def extract_numeric(value):
    # Keep only numeric characters and decimal point
    return ''.join(char for char in value if char.isdigit() or char == '.')

In [201]:
# Function to extract numeric values from the string
def extract_dimensions(value):
    # Find all numeric values with optional decimal points
    numbers = re.findall(r'\d+\.\d+|\d+', value)
    # Convert extracted values to float
    return [float(num) for num in numbers]

In [202]:
# Apply the extraction function
train_data['entity_value'] = train_data['entity_value'].apply(extract_dimensions)

In [203]:
# Apply the extraction function
print(train_data)
T_D = train_data

    index                                         image_link  group_id  \
0       1  https://m.media-amazon.com/images/I/71Zy+qaX82...         3   
1       2  https://m.media-amazon.com/images/I/619xuNVks4...         4   
2       3  https://m.media-amazon.com/images/I/61+uFY1+-d...         4   
3       4  https://m.media-amazon.com/images/I/413lP0yIzL...         1   
4       5  https://m.media-amazon.com/images/I/41KG3OYJo0...         3   
5       6  https://m.media-amazon.com/images/I/81vqHyDCSC...         2   
6       7  https://m.media-amazon.com/images/I/71GLMJ7TQi...         2   
7       8  https://m.media-amazon.com/images/I/41uC6f6q+B...         2   
8       9  https://m.media-amazon.com/images/I/41aIgv8HTE...         2   
9      10  https://m.media-amazon.com/images/I/71fhaUR7VG...         2   
10     11  https://m.media-amazon.com/images/I/21q4I7ziVq...         2   
11     12  https://m.media-amazon.com/images/I/61MV+cJP9U...         6   
12     13  https://m.media-amazon.com/

In [204]:
import pandas as pd
import re
import numpy as np

# Sample DataFrame (replace this with your actual data)
train_data = pd.DataFrame({
    'entity_value': ['165.1 MM * 175.6MM * 8.9 MM', '39.6 cm', '20.4 kg', '15.2 g']
})

# Function to extract dimensions or handle non-dimension entries
def extract_or_handle(value):
    numbers = re.findall(r'\d+\.\d+|\d+', value)
    numbers = [float(num) for num in numbers]
    if len(numbers) == 3:
        return numbers  # Return as list if 3 dimensions
    else:
        return np.nan  # Return NaN for non-dimension entries

# Apply the function
train_data['entity_value'] = train_data['entity_value'].apply(extract_or_handle)

# Check the result
print(train_data)



          entity_value
0  [165.1, 175.6, 8.9]
1                  NaN
2                  NaN
3                  NaN


In [205]:
# Train the model on the preprocessed images
model.fit(X_train_images, y_train, epochs=10, batch_size=16)

Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 104110.6953
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - loss: 97772.0859
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - loss: 89565.0156
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 367ms/step - loss: 79719.2500
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 409ms/step - loss: 70411.4297
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 388ms/step - loss: 66702.6016
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 405ms/step - loss: 72676.0547
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 624ms/step - loss: 73755.9297
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 379ms/step - loss: 69857.0312
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 591ms/step - loss: 66069.6094


In [206]:
predictions = model.predict(X_test_images)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step


In [207]:
def format_predictions(predictions, entity_name):
    formatted_predictions = []
    for pred in predictions:
        if entity_name == 'item_weight':
            formatted_predictions.append(f"{pred[0]:.2f} kg")  # Example: weight in kilograms
        elif entity_name == 'item_dimension':
            formatted_predictions.append(f"{pred[0]:.2f} cm")  # Example: dimensions in centimeters
    return formatted_predictions

In [208]:
test_predictions = format_predictions(predictions, test_data['entity_name'].iloc[0])

In [209]:
test_df = pd.DataFrame({'index': test_data['index'], 'prediction': test_predictions})

In [210]:
!pip install pytesseract
import matplotlib.pyplot as plt
import PIL
import pytesseract
import re
%matplotlib inline



# New Section

In [211]:
   print(test_df)

   index prediction
0      1  226.64 cm
1      2  142.93 cm
2      3  206.14 cm
3      4  211.72 cm
4      5  201.08 cm


In [212]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import numpy as np

In [219]:
T_D_split, val_data_split = train_test_split(T_D, test_size=0.2, random_state=42)

In [220]:
print(train_data.columns)
print(test_data.columns)
print(train_data)

Index(['entity_value'], dtype='object')
Index(['index', 'image_link', 'group_id', 'entity_name'], dtype='object')
          entity_value
0  [165.1, 175.6, 8.9]
1                  NaN
2                  NaN
3                  NaN


In [221]:
X_train = T_D_split['image_link']  # Features (image links)
y_train = T_D_split['entity_value']  # Target variable (actual values)
X_val = val_data_split['image_link']  # Validation features
y_val = val_data_split['entity_value']

In [222]:
y_pred = y_val.apply(lambda x: x)  # Dummy prediction to match ground truth

# Step 3: Format both predictions and actual values
def extract_numeric_string(val):
    """Helper function to standardize the format of predictions and ground truth"""
    if isinstance(val, str):
        return val.strip().lower()
    return str(val)

In [223]:
y_val = y_val.apply(extract_numeric_string)
y_pred = np.array([extract_numeric_string(val) for val in y_pred])

In [224]:
f1 = f1_score(y_val, y_pred, average='micro')  # Use 'micro' for multi-class problems
print(f"F1 Score: {f1}")

F1 Score: 1.0
